In [3]:
import os
from starlette.config import Config
from authlib.integrations.starlette_client import OAuth
from starlette.middleware.sessions import SessionMiddleware
from fastapi import Request
from starlette.responses import RedirectResponse
from authlib.integrations.starlette_client import OAuthError
from starlette.responses import HTMLResponse


config_data = {'GOOGLE_CLIENT_ID': '456154860004-v6j31iem46hni8lcj204p7r72rqchhjp.apps.googleusercontent.com', 'GOOGLE_CLIENT_SECRET': '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'}
starlette_config = Config(environ=config_data)
oauth = OAuth(starlette_config)
oauth.register(
    name='google',
    server_metadata_url='https://accounts.google.com/.well-known/openid-configuration',
    client_kwargs={'scope': 'openid email profile'},
)
app.add_middleware(SessionMiddleware, secret_key='kqDU1WL8n4jC3PpD2_j6OKvMDyxAIOEl46r3pNis')

@app.route('/login')
async def login(request: Request):
    redirect_uri = request.url_for('auth')  # This creates the url for the /auth endpoint
    return await oauth.google.authorize_redirect(request, redirect_uri)


@app.route('/auth')
async def auth(request: Request):
    try:
        access_token = await oauth.google.authorize_access_token(request)
    except OAuthError:
        return RedirectResponse(url='/')
    user_data = await oauth.google.parse_id_token(request, access_token)
    request.session['user'] = dict(user_data)
    return RedirectResponse(url='/')

@app.get('/')
def public(request: Request):
    user = request.session.get('user')
    if user:
        name = user.get('name')
        return HTMLResponse(f'<p>Hello {name}!</p><a href=/logout>Logout</a>')
    return HTMLResponse('<a href=/login>Login</a>')


@app.route('/logout')
async def logout(request: Request):
    request.session.pop('user', None)
    return RedirectResponse(url='/')

NameError: name 'app' is not defined

In [2]:
import secrets
generated_key = secrets.token_urlsafe(30)
print(generated_key)

kqDU1WL8n4jC3PpD2_j6OKvMDyxAIOEl46r3pNis
